# Trip Planner: API World vs MCP World 🌍✈️

This notebook walks through a **complex but intuitive example** to explain:

> **How classic APIs differ from MCP-style tools** when you build a trip-planning assistant.

Scenario we will use:

> *“I’m travelling from Hyderabad to London. What’s the expected weather, and what rough daily budget in INR should I plan for?”*

To answer this, we actually need to:

1. Get a **weather forecast** for London (via a real weather API).
2. Estimate a **daily travel budget** in GBP for a given traveler type (budget/standard/premium).
3. Convert that budget from **GBP → INR** via a real FX API.
4. Present everything in a clean, human-readable summary.

We will do this in two styles:

- **API World** → you manually orchestrate REST calls and glue data.
- **MCP World** → the same capabilities are exposed as high-level **tools** an LLM/agent can discover and call.


## 0. Setup (Colab-friendly)

If you are running this in **Google Colab**:
1. Upload the repo folder or mount it from Drive.
2. Run the following cell once to install dependencies.


In [ ]:
# If running in Colab, uncomment the next two lines after placing this notebook
# inside the repository root (complex-api-vs-mcp-trip-planner).

# !pip install -r ../requirements.txt

import os, sys
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.append(repo_root)
print("Repo root added to sys.path:", repo_root)


## Part A – API World: Manual Orchestration 🌐

In the **API world**:

- You must know each endpoint (weather, FX).
- You must know the query parameters for each.
- You must parse each JSON response.
- You must manually decide the *order* of calls.
- You must glue everything into a final message.

The function `plan_trip_api_style` in `src.api_world.planner_api_style` does exactly that.

We’ll call it for a fixed trip:
- Origin: Hyderabad
- Destination: London
- Traveler type: `standard`
- Home currency: INR
- 3 days of weather


In [ ]:
from src.api_world.planner_api_style import plan_trip_api_style

origin = "Hyderabad"
dest = "London"
traveler_type = "standard"  # 'budget' | 'standard' | 'premium'

print("[API WORLD] Manual orchestration result:\n" + "-" * 60)
print(
    plan_trip_api_style(
        origin_city=origin,
        dest_city=dest,
        traveler_type=traveler_type,
        home_currency="INR",
        days_of_weather=3,
    )
)


### Reflection: what did *you* (the developer) have to do in API world?

If you inspect the code in `src/api_world/` you’ll see that you (the human):

- Hard-coded a city→coordinates mapping for weather.
- Chose the **Open-Meteo** endpoint and the exact daily fields to request.
- Chose the **exchangerate.host** endpoint and understood its response.
- Decided the traveler-type budget table in GBP.
- Called these APIs in the right order.
- Wrote custom glue code to format a final answer.

From an LLM’s viewpoint, none of this is visible as *capabilities*.
It just sees one opaque Python function that you wired by hand.


## Part B – MCP World: Tool-Based Orchestration 🤖

In the **MCP world**, we expose the same logic as **discoverable tools**.

The MCP server (`src.mcp_world.mcp_server`) defines tools such as:

- `get_weather(city, days)` – high-level weather summary.
- `estimate_daily_budget(city, traveler_type, home_currency)` – FX + budget.
- `plan_trip_summary(origin_city, dest_city, traveler_type, home_currency, days_of_weather)` – orchestrates both.

An MCP-aware client (e.g., ChatGPT, Claude Desktop, or your own agent runner) can:

1. **Discover** these tools with `list_tools()`.
2. Decide *which* tool to call based on the user’s natural-language request.
3. Fill in the parameters and call the tool.
4. Take the JSON output and generate a nice final narrative.

Below we *simulate* part of this: introspecting tools and calling `plan_trip_summary` directly.


In [ ]:
import json, textwrap
from src.mcp_world.mcp_server import mcp, plan_trip_summary

# Introspect tools (similar to what an MCP client would see)
tools = mcp._server_state.tools  # internal, OK for teaching

print("[MCP WORLD] Tools exposed by TripPlannerMCP:\n" + "-" * 60)
for name, tool in tools.items():
    print(f"Tool: {name}")
    if tool.description:
        print(f"  Description: {tool.description}")
    print("  Parameters schema:")
    print(textwrap.indent(json.dumps(tool.parameters, indent=2), "    "))
    print()


In [ ]:
# Simulate what an LLM might do: pick plan_trip_summary as the right tool and call it.

origin = "Hyderabad"
dest = "London"
traveler_type = "standard"  # same as before

print("[MCP WORLD] Simulated LLM decision:")
print(
    f'  → Call plan_trip_summary(origin_city="{origin}", '
    f'dest_city="{dest}", traveler_type="{traveler_type}")'
)

result = plan_trip_summary(
    origin_city=origin,
    dest_city=dest,
    traveler_type=traveler_type,  # type: ignore[arg-type]
    home_currency="INR",
    days_of_weather=3,
)

print()
print("[MCP WORLD] Raw JSON result from tool (what the model sees):")
print(json.dumps(result, indent=2))

print()
print("[MCP WORLD] Example final answer an LLM could generate:")
print(
    textwrap.dedent(
        f"""\
        For your trip from {result['origin_city']} to {result['dest_city']} as a {result['traveler_type']} traveler:\n\n"
        f"        Weather outlook (next {result['days_of_weather']} days):\n"
        f"        {result['weather_summary']}\n\n"
        f"        Budget estimation:\n"
        f"        - Base daily cost in {result['dest_city']}: {result['dest_daily_cost']:.0f} {result['dest_currency']}\n"
        f"        - Approx. daily budget in {result['home_currency']}: {result['converted_daily_cost']:.0f} {result['home_currency']}\n\n"
        f"        You can adjust the traveler type (budget / standard / premium) to explore different budget levels.\n"
        """
    ).strip()
)


## Part C – API vs MCP: Interface Comparison

Look carefully at the **interfaces** you just used:

### API World

- You called `plan_trip_api_style(...)`.
- Inside, it manually:
  - hit the weather API
  - hit the FX API
  - glued results into a string.
- To add another capability (e.g., hotel recommendations), you’d wire **more APIs** inside this function.
- From an LLM’s perspective, this is a **black box**.

### MCP World

- You have **separate tools**:
  - `get_weather` – capability: weather lookup
  - `estimate_daily_budget` – capability: cost estimation
  - `plan_trip_summary` – composed capability: full trip plan
- Each tool **self-describes**:
  - name
  - parameters (with types)
  - description
- A model can:
  - discover tools with `list_tools()`
  - choose which tool to call based on user intent
  - chain multiple tools in a reasoning loop (agentic behavior).

### The Big Idea

> **APIs** are low-level pipes for humans.
>
> **MCP tools** are high-level capabilities for models.

In the API world, **you** are the orchestrator.
In the MCP world, **the model** can become the orchestrator, as long as you provide good tools.


## Part D – Play with Different Scenarios (Optional)

Try changing:

- `traveler_type` → `"budget"` or `"premium"`.
- Destination city (for which you add entries in the budget & city registries in code).

Then re-run the API and MCP cells and observe:

- How much code you touch in API world.
- How clean it feels to keep adding **new tools** in MCP world.
